In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Medical_knowledge_base.csv', encoding='ISO-8859-1')
df.head()


,name,desc,prevent,cause,symptom,acompany,cure_department,cure_way
0,alveolar proteinosis,"alveolar proteinosis (pap), also known as rose...","1, to avoid infection with mycobacterial disea...","the cause is unknown, and it is speculated tha...","['cyanosis', 'chest pain', 'breathing difficul...",['multiple lung infections'],"['internal medicine', 'respiratory medicine']",['bronchoalveolar lavage']
1,pertussis,pertussis (whoopingcough) is an acute respirat...,"1, control of the source of infection: in the ...",(1) causes of the disease the pathogen is b. p...,"['hanging sound when inhaling', 'sexual cough'...",['lung atelect'],"['pediatrics', 'pediatric medicine']","['medical treatment', 'supportive treatment']"
2,benzene poisoning,benzene is an aromatic hydrocarbon compound ob...,"for patients with acute poisoning, you can imm...",inhalation of benzene vapor or skin contact wi...,"['disgusting', 'twitching', 'feeling obstacle']",['anemia'],['emergency department'],"['medical treatment', 'supportive treatment']"
3,asthmatic bronchitis,"asthmatoid bronchitis, also known as asthmatic...","according to the above section, for children w...",because a variety of viral and bacterial infec...,"['shrubbing's wheezing', 'wheeling sound', 'ci...",['bronchial asthma'],"['internal medicine', 'respiratory medicine']","['medical treatment', 'supportive treatment']"
4,adult respiratory distress syndrome,"adult respiratory distress syndrome, abbreviat...",patients with high-risk should be closely obse...,suppurative infections can cause bacterial tox...,"['difficulty breathing', 'purple', 'cardiac re...",['bacterial pneumonia'],"['internal medicine', 'respiratory medicine']","['medical treatment', 'supportive treatment']"


In [4]:
! wget https://raw.githubusercontent.com/deepmipt/DeepPavlov/squad_multilingual_configs/deeppavlov/configs/squad/squad_bert_multilingual_freezed_emb.json

--2023-03-06 16:19:48--  https://raw.githubusercontent.com/deepmipt/DeepPavlov/squad_multilingual_configs/deeppavlov/configs/squad/squad_bert_multilingual_freezed_emb.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3538 (3.5K) [text/plain]
Saving to: ‘squad_bert_multilingual_freezed_emb.json’

squad_bert_multilin 100%[===================>]   3.46K  --.-KB/s    in 0s      

2023-03-06 16:19:48 (56.0 MB/s) - ‘squad_bert_multilingual_freezed_emb.json’ saved [3538/3538]



In [5]:
! pip install -q deeppavlov

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.7/433.7 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.6 MB/s eta 0:00:00


In [6]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.9 MB/s eta 0:00:00


In [7]:
from deeppavlov import build_model, configs

# Load the question-answering model
model = build_model(configs.squad.squad_bert, download=True)


2023-03-06 16:20:24.41 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz to /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz to /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz
100%|██████████| 1.18G/1.18G [00:54<00:00, 21.7MB/s]
2023-03-06 16:21:18.994 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz archive into /root/.deeppavlov/models/squad_torch_bert/cased/bert-base-cased
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz archive into /root/.deeppavlov/models/squad_torch_bert/cased/bert-base-cased


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [8]:
model(['alveolar proteinosis (pap), also known as rosen-castle-man-liebow syndrome, is a rare disease.the disease is filled with pas staining positive in the alveolar and bronchial lumens, and the phospholipid-rich protein substance from the lung is characterized by the fact that it occurs in young and middle-aged people, and the incidence of males is about three times that of females.'], 
      ['cyanosis', 'chest pain', 'breathing difficulties', 'living'])

[['alveolar proteinosis'], [0], [0.9983924627304077]]

In [9]:
# Combine the description and symptoms into a single input string
df['input_text'] = df['desc'] + ' .And I have this symptoms: ' + df['symptom']

In [14]:
# Define a function to handle user input and return the model's response

def get_bot_response(user_input):
    # Use the question-answering model to generate a response
    response = model([user_input], df['input_text'])
    
    # Extract the answer and the corresponding medical condition
    answer = response[0][0]
    condition = df.iloc[response[1][0]]['desc']
    
    # Extract the cure_way for the medical condition
    cure_way = df.loc[df['desc'] == condition]['cure_way'].values[0]
    
    # Return the answer and cure_way as a tuple
    return answer, cure_way


In [13]:
# Use a loop to keep the chatbot running
while True:
    user_input = input('You: ')
    if user_input.lower() == 'exit':
        break
    bot_response = get_bot_response(user_input)
    print('Bot:', bot_response)

You: 'alveolar proteinosis (pap), also known as rosen-castle-man-liebow syndrome,\  is a rare disease.the disease is filled with pas staining positive in the alveolar\   and bronchial lumens, and the phospholipid-rich protein substance from the lung is\    characterized by the fact that it occurs in young and middle-aged people, and the \    incidence of males is about three times that of females.'' .And I have this symptoms: ' 'cyanosis', 'chest pain', 'breathing difficulties', 'living'
Bot: ('females', "['surgical treatment', 'supportive treatment', 'rehabilitation treatment']")
You: pertussis (whoopingcough) is an acute respiratory infection caused by bordetella pertussis.it is characterized by paroxysmal spasmodic cough, with a special snoring inhalation snoring at the end of the cough.the course of the disease is long, up to several weeks or even 3 months, so it is called pertussis.more common in children under the age of 5, young children suffering from this disease are prone to 